In [1]:
%matplotlib inline

# Predicting Titanic Victims
In this notebook we create a neural network to predict the survival rate of a Titanic passanger using properties from their trip. These can be things like the fare, gender, cabin, etc.

In [96]:
import random
import numpy as np
import pandas as pd

In [85]:
from keras import models
from keras import layers
from keras import optimizers
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.utils.np_utils import to_categorical

In [9]:
dataset = pd.read_csv('data/data.csv', quotechar='"')

In [10]:
dataset.head(2)

,Id,Pclass,Sex,Parch,Cabin,Embarked,Fare,Age,SibSp,Survived
0,1,3,1,0,0,3,7.2500,22.0,1,0
1,2,1,0,0,3,1,71.2833,38.0,1,1


In [80]:
def prepare_dataset(df):
    """
    Transforms a pandas DataFrame into a training
    tensor (X) and a prediction vector (Y).
    
    Paraemters
    ----------
    df: pandas DataFrame
        Pandas DataFrame with the expected
        predictor variables.
    
    Returns
    -------
    X, Y: np.array
        Both a prediction variables and predicted
        variables represented as np.arrays
    """
    pclass = df['Pclass'].tolist()
    sex = df['Sex'].tolist()
    parch = df['Parch'].tolist()
    cabin = df['Cabin'].tolist()
    embarked = df['Embarked'].tolist()
    
    # get fare list ready !
    fare = df['Fare'].tolist()
    max_fare = max(fare)
    fare = [x / max_fare for x in fare]
    
    # get age ready !
    age = df['Age'].tolist()
    max_age = max(age)
    age = [x / max_age for x in age]
    
    # get number of sib & spouses ready !
    sib_sp = df['SibSp'].tolist()
    max_sib_sp = max(sib_sp)
    sib_sp = [x / max_sib_sp for x in sib_sp]
    
    Y = np.array(df['Survived'].tolist())
    
    X = []
    for i in range(0, len(Y)):
        X.append(
            np.array([
                pclass[i],
                sex[i],
                parch[i],
                cabin[i],
                embarked[i],
                fare[i],
                age[i],
                sib_sp[i]
            ])
        )
    
    X = np.array(X).reshape(len(Y), X[0].shape[0])
    Y = Y.reshape(len(Y), 1)
    return X, Y

In [81]:
X, Y = prepare_dataset(dataset)

Let's now create a Keras model. The current model only uses a series of `Dense()` layers. The pattern of organizing them (from a layer with many units to one with fewer units and then many units again) is called an `autoencoder`. I have no idea if this is good or bad for this problem, so feel free to change it around as you want. 

In [108]:
K.clear_session()
model = models.Sequential([
    
    layers.Dense(64, input_dim=X.shape[1]),
    layers.Activation('relu'),
    
#     layers.Dropout(0.2),
    
    layers.Dense(32),
    layers.Activation('relu'),
    
#     layers.Dropout(0.2),
    
    layers.Dense(16),
    layers.Activation('relu'),
    
    layers.Dense(8),
    layers.Activation('relu'),
    
    layers.Dense(4),
    layers.Activation('relu'),
    
    layers.Dense(8),
    layers.Activation('relu'),
    
    layers.Dense(16),
    layers.Activation('relu'),
    
    layers.Dense(32),
    layers.Activation('relu'),
    
    layers.Dense(1),
    layers.Activation('sigmoid')
    
])

optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                576       
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
activation_2 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                528       
_________________________________________________________________
activation_3 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 136       
__________

Finanlly, let's train the network. We will be following the process using TensorBoard (use the command `tensorboard --logdir=logs/`). The following snippet fits the network and generates the TensorBoard log ouput. Note that it will generate training logs in different folders using a hashing function. We do that just to make sure that every run is different and that it doesn't overwrite the previous run logs.

In [107]:
hash = random.getrandbits(128)
hex_code = '%032x' % hash
model_name = 'first_try_{}'.format(hex_code[:6])
tensorboard = TensorBoard(log_dir='./logs/{}'.format(model_name))
model.fit(X, Y, epochs=10**3, batch_size=10, callbacks=[tensorboard])

Epoch 1/1000
891/891 [==============================] - 0s 262us/step - loss: 0.5792 - acc: 0.6745
Epoch 2/1000
891/891 [==============================] - 0s 130us/step - loss: 0.5144 - acc: 0.7329
Epoch 3/1000
891/891 [==============================] - 0s 130us/step - loss: 0.4855 - acc: 0.7789
Epoch 4/1000
891/891 [==============================] - 0s 138us/step - loss: 0.4765 - acc: 0.7834
Epoch 5/1000
891/891 [==============================] - 0s 131us/step - loss: 0.4695 - acc: 0.7912
Epoch 6/1000
891/891 [==============================] - 0s 131us/step - loss: 0.4616 - acc: 0.8036
Epoch 7/1000
891/891 [==============================] - 0s 129us/step - loss: 0.4556 - acc: 0.8058
Epoch 8/1000
891/891 [==============================] - 0s 128us/step - loss: 0.4490 - acc: 0.8058
Epoch 9/1000
891/891 [==============================] - 0s 129us/step - loss: 0.4518 - acc: 0.8047
Epoch 10/1000
891/891 [==============================] - 0s 130us/step - loss: 0.4473 - acc: 0.8092
Epoch 11/